# Realestate Rental ROI Prediction - Loading Dataset using Snowpark Python


## Loading Real Estates Ads into Snowflake

### Import the dependencies and connect to Snowflake

In [1]:
# Snowpark
# Print the version of Snowpark we are using
from importlib.metadata import version

from snowflake.snowpark import Session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *

version("snowflake_snowpark_python")

'0.12.0'

In [2]:
# Other
import json

**Before connecting make sure you have updated creds.json with information for your Snowflake account**

In [3]:
with open("creds.json") as f:
    connection_parameters = json.load(f)

In [4]:
session = Session.builder.configs(connection_parameters).create()

The **get_** functions can be use to get information about the current database, schema, role etc

In [5]:
print(
    f"Current schema: {session.get_fully_qualified_current_schema()}, current role: {session.get_current_role()}, current warehouse:  {session.get_current_warehouse()}"
)

Current schema: "SNOWPARKDEMO_DB"."PUBLIC", current role: "SNOWPARKDEMO_ROLE", current warehouse:  "SNOWPARKDEMO_WH"


### Define Staging Area and the Schema for the transaction table

Using SQL we can create a internal stage and then use the **put** function to uplad the **realestatesads.csv.gz** file to it.

In [6]:
stage_name = "S3_SNOW_REALESTATEFR"

# Create a internal staging area for uploading the source file
#session.sql(f"CREATE or replace STAGE {stage_name}").collect()

# Upload the source file to the stage
#putResult = session.file.put(
#    "data/realestatesads.csv.gz", f"@{stage_name}", auto_compress=False
#)

#putResult

session.sql(f"ls @{stage_name}/CSV").collect()

[Row(name='s3://realestatefrance/CSV/realestatesads.csv.gz', size=577459423, md5='655fd3cf27a155eeef6f24193c5e7f73', last_modified='Thu, 12 Jan 2023 14:42:31 GMT'),
 Row(name='s3://realestatefrance/CSVCOMPRESSS/2012/data_01a99a39-3201-7ece-0000-8f7d01e78a82_115_1_0.csv.gz', size=296, md5='f183c1ac62c1815686ffe5841b060669', last_modified='Thu, 12 Jan 2023 15:22:38 GMT'),
 Row(name='s3://realestatefrance/CSVCOMPRESSS/2013/data_01a99a39-3201-7ece-0000-8f7d01e78a82_115_6_0.csv.gz', size=240, md5='bccae3c36151b9d862850f33f067f99f', last_modified='Thu, 12 Jan 2023 15:22:38 GMT'),
 Row(name='s3://realestatefrance/CSVCOMPRESSS/2014/data_01a99a39-3201-7ece-0000-8f7d01e78a82_415_4_0.csv.gz', size=3719, md5='5ec87adc64551c08d0c2e1555d26f558', last_modified='Thu, 12 Jan 2023 15:22:38 GMT'),
 Row(name='s3://realestatefrance/CSVCOMPRESSS/2015/data_01a99a39-3201-7ece-0000-8f7d01e78a82_015_3_0.csv.gz', size=23507, md5='46ff0e2d597ad51a59a9eff31fcc619b', last_modified='Thu, 12 Jan 2023 15:22:38 GMT'),


Define the schma for our **ADS** table

In [7]:
# Define the schema for the ADS table
df_ADS_Schema = StructType(
    [
        StructField("ADS_ID", IntegerType()),
        StructField("ADS_CATEGORY_NAME", StringType()),
        StructField("ADS_FIRST_PUBLICATION_DATE", DateType()),
        StructField("ADS_SUBJECT", StringType()),
        StructField("ADS_PRICE", IntegerType()),
        StructField("ADS_OPT_URGENT", BooleanType()),
        StructField("ADS_OWNER_TYPE", StringType()),
        StructField("ADS_ATTR_REAL_ESTATE_TYPE", StringType()),
        StructField("ADS_ATTR_ROOMS", IntegerType()),
        StructField("ADS_ATTR_SQUARE", IntegerType()),
        StructField("ADS_ATTR_GES", StringType()),
        StructField("ADS_ATTR_ENERGY_RATE", StringType()),
        StructField("ADS_ATTR_FURNISHED", StringType()),
        StructField("ADS_GEO_LAT", FloatType()),
        StructField("ADS_GEO_LNG", FloatType()),
        StructField("ADS_GEO_CITY_LAT", FloatType()),
        StructField("ADS_GEO_CITY_LNG", FloatType()),
        StructField("ADS_GEO_CITY", StringType()),
        StructField("ADS_GEO_ZIPCODE", StringType()),
        StructField("ADS_GEO_REGION", StringType()),
        StructField("ADS_GEO_DEPARTEMENT", StringType()),
        StructField("ADS_GEO_ARRONDISSEMENT", StringType()),
        StructField("ADS_GEO_ARRONDISSEMENT_LAT", FloatType()),
        StructField("ADS_GEO_ARRONDISSEMENT_LNG", FloatType()),
    ]
)

# Crete a reader
df_Reader = session.read.schema(df_ADS_Schema)


Load the **realestatesads.csv.gz** to a DataFrame reader and save into a table

In [8]:
# Get the data into the data frame
df_ADS = df_Reader.csv("@S3_SNOW_REALESTATEFR/CSVUNCOMPRESSS")
# Write the dataframe in a table
ret = df_ADS.write.mode("overwrite").saveAsTable("ANNONCES_IMMOBILIERES_CSVUNCOMPRESSS")
ret

In [9]:
# Get the data into the data frame
df_ADS = df_Reader.csv("@S3_SNOW_REALESTATEFR/CSVCOMPRESSS")
# Write the dataframe in a table
ret = df_ADS.write.mode("overwrite").saveAsTable("ANNONCES_IMMOBILIERES_CSVCOMPRESS")
ret

In [10]:
# Get the data into the data frame
df_ADS = df_Reader.csv("@S3_SNOW_REALESTATEFR/CSVSplit/*.csv.gz")
# Write the dataframe in a table
ret = df_ADS.write.mode("overwrite").saveAsTable("ANNONCES_IMMOBILIERES_CSVSPLIT")
ret

Chek the result